In [1]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from google.colab import files
uploaded = files.upload()

Saving train_pronto.csv to train_pronto.csv


In [0]:
df = uploaded

In [0]:
df = pd.read_csv('train_pronto.csv')

In [0]:
df.drop(['Unnamed: 0', 'weekend'], axis = 1, inplace = True)

In [0]:
class_name = 'Occupancy'
attributes = [col for col in df.columns if col != class_name]
X = df[attributes].values
y = df[class_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)

In [0]:
def build_model():
    
    n_feature = X_train.shape[1]
    model = Sequential()
    model.add(Dense(128, input_dim=n_feature, activation='relu', kernel_initializer = 'uniform'))
    model.add(Dense(64, activation='tanh', kernel_initializer = 'uniform'))
    model.add(Dense(32, activation = 'tanh', kernel_initializer = 'uniform' ))
    model.add(Dense(4, activation = 'tanh', kernel_initializer = 'uniform' ))
    model.add(Dense(2, activation = 'sigmoid', kernel_initializer = 'uniform' ))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = 'uniform' ))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
model = build_model()

history = model.fit(X_train, y_train, epochs=50, batch_size=50).history

Epoch 1/50
11513/11513 [==============================] - 1s 60us/step - loss: 0.6169 - accuracy: 0.7683
Epoch 2/50
11513/11513 [==============================] - 0s 31us/step - loss: 0.4771 - accuracy: 0.7683
Epoch 3/50
11513/11513 [==============================] - 0s 33us/step - loss: 0.3952 - accuracy: 0.7683
Epoch 4/50
11513/11513 [==============================] - 0s 31us/step - loss: 0.3475 - accuracy: 0.7683
Epoch 5/50
11513/11513 [==============================] - 0s 30us/step - loss: 0.3141 - accuracy: 0.7683
Epoch 6/50
11513/11513 [==============================] - 0s 30us/step - loss: 0.2831 - accuracy: 0.7683
Epoch 7/50
11513/11513 [==============================] - 0s 30us/step - loss: 0.2634 - accuracy: 0.7683
Epoch 8/50
11513/11513 [==============================] - 0s 29us/step - loss: 0.2481 - accuracy: 0.9159
Epoch 9/50
11513/11513 [==============================] - 0s 30us/step - loss: 0.2318 - accuracy: 0.9511
Epoch 10/50
11513/11513 [==============================

In [10]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('modello no reg', 'Loss %f, Accuracy %f' % (test_loss, test_acc))

4935/4935 [==============================] - 0s 23us/step
modello no reg Loss 0.066624, Accuracy 0.983181


In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
es = EarlyStopping(monitor='val_loss', patience=15)
mc = ModelCheckpoint('best_model_NOREG.h5', monitor='val_loss', save_best_only=True)

history_y = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, 
                      batch_size=50, callbacks=[es,mc]).history

Train on 11513 samples, validate on 4935 samples
Epoch 1/100
11513/11513 [==============================] - 0s 41us/step - loss: 0.0583 - accuracy: 0.9859 - val_loss: 0.0689 - val_accuracy: 0.9807
Epoch 2/100
11513/11513 [==============================] - 0s 41us/step - loss: 0.0525 - accuracy: 0.9887 - val_loss: 0.0506 - val_accuracy: 0.9895
Epoch 3/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.0532 - accuracy: 0.9883 - val_loss: 0.0508 - val_accuracy: 0.9882
Epoch 4/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.0544 - accuracy: 0.9871 - val_loss: 0.0681 - val_accuracy: 0.9822
Epoch 5/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.0516 - accuracy: 0.9879 - val_loss: 0.0577 - val_accuracy: 0.9848
Epoch 6/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.0538 - accuracy: 0.9875 - val_loss: 0.0475 - val_accuracy: 0.9895
Epoch 7/100
11513/11513 [==============================] - 0s 3

In [0]:
# REGULARIZATION

In [17]:
from keras.regularizers import l2
def build_L2_model():
    n_feature = X_train.shape[1]
    model = Sequential()
    model.add(Dense(128, input_dim=n_feature, activation='relu', kernel_initializer = 'uniform', kernel_regularizer=l2(0.5)))
    model.add(Dense(64, activation='tanh', kernel_initializer = 'uniform'))
    model.add(Dense(32, activation = 'tanh', kernel_initializer = 'uniform'))
    model.add(Dense(4, activation = 'tanh', kernel_initializer = 'uniform' , kernel_regularizer=l2(0.01) ))
    model.add(Dense(2, activation = 'sigmoid', kernel_initializer = 'uniform' ))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = 'uniform' ))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
mc = ModelCheckpoint('best_model_L2.h5', monitor='val_loss', save_best_only=True)

L2_model = build_L2_model()
h_L2 = L2_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, 
                    batch_size=50, callbacks=[es, mc]).history

Train on 11513 samples, validate on 4935 samples
Epoch 1/100
11513/11513 [==============================] - 1s 55us/step - loss: 0.6759 - accuracy: 0.7589 - val_loss: 0.5859 - val_accuracy: 0.7682
Epoch 2/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.5611 - accuracy: 0.7683 - val_loss: 0.5483 - val_accuracy: 0.7682
Epoch 3/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.4693 - accuracy: 0.7683 - val_loss: 0.4051 - val_accuracy: 0.7682
Epoch 4/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.3804 - accuracy: 0.7683 - val_loss: 0.3530 - val_accuracy: 0.7682
Epoch 5/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.3371 - accuracy: 0.7683 - val_loss: 0.3485 - val_accuracy: 0.7682
Epoch 6/100
11513/11513 [==============================] - 0s 37us/step - loss: 0.3082 - accuracy: 0.7683 - val_loss: 0.2889 - val_accuracy: 0.7682
Epoch 7/100
11513/11513 [==============================] - 0s 3

In [0]:
from keras.layers import Dropout

In [19]:
def build_DROPOUT_model():
    n_feature = X_train.shape[1]
    model = Sequential()
    model.add(Dense(128, input_dim=n_feature, activation='relu', kernel_initializer = 'uniform', kernel_regularizer=l2(0.5)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='tanh', kernel_initializer = 'uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'tanh', kernel_initializer = 'uniform'))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation = 'tanh', kernel_initializer = 'uniform' , kernel_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation = 'sigmoid', kernel_initializer = 'uniform' ))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = 'uniform' ))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
mc = ModelCheckpoint('best_model_DROPOUT.h5', monitor='val_loss', save_best_only=True)

DROPOUT_model = build_DROPOUT_model()
h_DROPOUT = DROPOUT_model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                              epochs=100, batch_size=50, callbacks=[es,mc]).history

Train on 11513 samples, validate on 4935 samples
Epoch 1/100
11513/11513 [==============================] - 1s 66us/step - loss: 0.6821 - accuracy: 0.7346 - val_loss: 0.5907 - val_accuracy: 0.7682
Epoch 2/100
11513/11513 [==============================] - 1s 43us/step - loss: 0.5530 - accuracy: 0.7683 - val_loss: 0.4797 - val_accuracy: 0.7682
Epoch 3/100
11513/11513 [==============================] - 1s 44us/step - loss: 0.4611 - accuracy: 0.7683 - val_loss: 0.4050 - val_accuracy: 0.7682
Epoch 4/100
11513/11513 [==============================] - 1s 45us/step - loss: 0.4067 - accuracy: 0.7683 - val_loss: 0.3653 - val_accuracy: 0.7682
Epoch 5/100
11513/11513 [==============================] - 1s 46us/step - loss: 0.3757 - accuracy: 0.7683 - val_loss: 0.3409 - val_accuracy: 0.7682
Epoch 6/100
11513/11513 [==============================] - 1s 45us/step - loss: 0.3583 - accuracy: 0.7683 - val_loss: 0.3226 - val_accuracy: 0.7682
Epoch 7/100
11513/11513 [==============================] - 1s 4

In [20]:
# laod best models and test them
from keras.models import load_model

best_NOREG_model = load_model('best_model_NOREG.h5')
best_L2_model = load_model('best_model_L2.h5')
best_DROPOUT_model = load_model('best_model_DROPOUT.h5')

loss_NOREG, acc_NOREG = best_NOREG_model.evaluate(X_test, y_test)
loss_L2, acc_L2 = best_L2_model.evaluate(X_test, y_test)
loss_DROPOUT, acc_DROPOUT = best_DROPOUT_model.evaluate(X_test, y_test)
print('NO REG:', 'Loss %f, Accuracy %f' % (loss_NOREG, acc_NOREG))
print('REG:','Loss %f, Accuracy %f' % (loss_L2, acc_L2))
print('DROPOUT','Loss %f, Accuracy %f' % (loss_DROPOUT, acc_DROPOUT))

4935/4935 [==============================] - 0s 29us/step
NO REG: Loss 0.043998, Accuracy 0.990071
REG: Loss 0.058904, Accuracy 0.989463
DROPOUT Loss 0.058889, Accuracy 0.989260
